In [ ]:
import deepxde as dde
import numpy as np
from deepxde.backend import tf 
from scipy import io
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# 1. 读取 CSV 文件（假设文件有列标题）
df = pd.read_csv('./changhua_data.csv', encoding='utf-8-sig')

# 2. 提取输入特征和目标列
# 这里假设第2列是输入特征，第1列是目标值
col_x = df.iloc[:, 2].values.reshape(-1, 1)[0:100]  # 第2列 (输入特征)
col_y = df.iloc[:, 1].values.reshape(-1, 1)[0:100]  # 第1列 (目标)

# 3. 划分训练集和测试集，80%训练，20%测试
X_train, X_test, y_train, y_test = train_test_split(col_x, col_y, test_size=0.2, random_state=42)

# 4. 数据标准化（如果需要标准化）
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_x.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)

X_test_scaled = scaler_x.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

# 5. 创建 DeepXDE 的 DataSet
data = dde.data.DataSet(
    X_train=X_train_scaled,
    y_train=y_train_scaled,
    X_test=X_test_scaled,
    y_test=y_test_scaled
)

# 6. 检查数据
print("训练数据输入 X_train:", X_train_scaled[:5])
print("训练数据输出 y_train:", y_train_scaled[:5])
print("测试数据输入 X_test:", X_test_scaled[:5])
print("测试数据输出 y_test:", y_test_scaled[:5])



In [ ]:
layer_size = [1] + [50] * 3 + [1]
activation = "tanh"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

In [ ]:
model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["l2 relative error"])
losshistory, train_state = model.train(iterations=50000)

In [ ]:
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

In [ ]:
# 1. 读取CSV文件
df = pd.read_csv("./changhua_data.csv")

# 2. 提取并处理时间列 (第1列)
# 使用 pandas 将时间转换为 datetime 对象
df['time'] = pd.to_datetime(df.iloc[:, 0])

# 将时间转换为数值形式，例如时间戳（秒）
x = df['time'].astype(np.int64) // 10**9  # 转换为秒级时间戳
x = x.values.reshape(-1, 1).astype(np.float32)

# 3. 分支网络输入：气象站的降雨数据 (第3到第9列)
v = df.iloc[:, 2:9].values.astype(np.float32)

# 4. 目标：流量 (第2列)
y = df.iloc[:, 1].values.reshape(-1, 1).astype(np.float32)

# 5. 使用 train_test_split 划分数据集为 80% 训练集和 20% 测试集
X_train_branch, X_test_branch, X_train_trunk, X_test_trunk, y_train, y_test = train_test_split(
    v, x, y, test_size=0.2, random_state=42
)

# 6. 构建X_train为一个元组，分别对应分支和主干网络的输入
X_train = (X_train_branch, X_train_trunk)
X_test = (X_test_branch, X_test_trunk)
print(X_train_branch.shape)
print(X_train_trunk.shape)
print(y_train.shape)
# len(X_test[1]) != y_test.shape[1]:
print(y_test.shape[1])
print(len(X_test[1]))
# X_train[0]) != y_train.shape[0]
print(len(X_train[1]))
print(y_train.shape[1])

In [ ]:
data = dde.data.TripleCartesianProd(
    X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test  # 假设你有测试数据
)

# 7. 现在你可以将这些数据用于 DeepONet 的输入

# 然后定义网络并训练
m = 100  # 假设有100个气象站输入
dim_x = 1  # 时间作为输入维度
net = dde.nn.DeepONetCartesianProd(
    [m, 40, 40],
    [dim_x, 40, 40],
    "relu",
    "Glorot normal",
)

model = dde.Model(data, net)
model.compile("adam", lr=0.001, metrics=["mean l2 relative error"])
losshistory, train_state = model.train(iterations=10000)


In [ ]:

# Load dataset
d = np.load("/Users/zjr/Downloads/antiderivative_aligned_train.npz", allow_pickle=True)
X_train = (d["X"][0].astype(np.float32), d["X"][1].astype(np.float32))
y_train = d["y"].astype(np.float32)
d = np.load("/Users/zjr/Downloads/antiderivative_aligned_test.npz", allow_pickle=True)
X_test = (d["X"][0].astype(np.float32), d["X"][1].astype(np.float32))
y_test = d["y"].astype(np.float32)

data = dde.data.TripleCartesianProd( 
    X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test
)

# Choose a network
m = 100
dim_x = 1
net = dde.nn.DeepONetCartesianProd(
    [m, 40, 40],
    [dim_x, 40, 40],
    "relu",
    "Glorot normal",
)

# Define a Model
model = dde.Model(data, net)

# Compile and Train
model.compile("adam", lr=0.001, metrics=["mean l2 relative error"])
losshistory, train_state = model.train(iterations=10000)

# Plot the loss trajectory
dde.utils.plot_loss_history(losshistory)
plt.show()

In [ ]:
import numpy as np
file_path="/Users/zjr/Downloads/antiderivative_aligned_test.npz"
poem=np.load(file_path,allow_pickle=True)
poem.files

In [ ]:
x,y = poem['X'],poem['y']

In [ ]:
print(type(x))

In [ ]:
print(x[0])

In [ ]:
print(x[1])

In [ ]:
print(y)

In [ ]:
print(x[0].shape)
print(x[1].shape)
print(y.shape)

In [ ]:
import numpy as np
import pandas as pd
import deepxde as dde
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('./changhua_data.csv')

# Extract columns
# Assume: First column is time, second column is flow, third to ninth are rainfall data
times = pd.to_datetime(data.iloc[:, 0].values)  # Convert time to datetime
flows = data.iloc[:, 1].values.reshape(-1, 1)
rainfalls = data.iloc[:, 2:].values

# Create sequences for time-series prediction
def create_sequences(data, target, input_steps, output_steps):
    X, y, times_seq = [], [], []
    for i in range(len(data) - input_steps - output_steps + 1):
        X.append(data[i:(i + input_steps), :])
        y.append(target[(i + input_steps):(i + input_steps + output_steps), 0])
        times_seq.append(times[i + input_steps:(i + input_steps + output_steps)])
    return np.array(X), np.array(y), np.array(times_seq)

input_steps = 12  # Use past 12 hours
output_steps = 6  # Predict next 6 hours

# Data preprocessing
scaler_flow = MinMaxScaler()
flows_scaled = scaler_flow.fit_transform(flows)

scaler_rainfall = MinMaxScaler()
rainfalls_scaled = scaler_rainfall.fit_transform(rainfalls)

# Combine rainfall and flow data for sequence creation
combined_data = np.concatenate((rainfalls_scaled, flows_scaled), axis=1)

# Create sequences
X, y, times_seq = create_sequences(combined_data, flows_scaled, input_steps, output_steps)

# Train-test split
X_train, X_test, y_train, y_test, times_train, times_test = train_test_split(X, y, times_seq, test_size=0.2, random_state=42)

# Define DeepONet model using DeepXDE
# branch = dde.maps.FNN([input_steps * (X_train.shape[2] - 1), 64, 32], "relu", "Glorot normal")
# trunk = dde.maps.FNN([input_steps, 64, 32], "relu", "Glorot normal")
# net = dde.maps.DeepONet(branch, trunk)

branch = dde.maps.FNN([input_steps * (X_train.shape[2] - 1), 64, 32], "relu", "Glorot normal")
trunk = dde.maps.FNN([input_steps, 64, 32], "relu", "Glorot normal")
net = dde.maps.DeepONet(branch, trunk, activation="relu", kernel_initializer="Glorot normal")

model = dde.Model(data=None, net=net)
model.compile("adam", lr=0.001, metrics=["mae"])

# Prepare data for DeepONet
branch_train = X_train[:, :, :-1].reshape(len(X_train), -1)
trunk_train = X_train[:, :, -1:].reshape(len(X_train), -1)
y_train_flat = y_train.reshape(-1, output_steps)

branch_test = X_test[:, :, :-1].reshape(len(X_test), -1)
trunk_test = X_test[:, :, -1:].reshape(len(X_test), -1)
y_test_flat = y_test.reshape(-1, output_steps)

# Train the model
losshistory, train_state = model.train([branch_train, trunk_train], y_train_flat, epochs=100, batch_size=16, validation_data=([branch_test, trunk_test], y_test_flat))

# Evaluate the model
evaluation = model.evaluate([branch_test, trunk_test], y_test_flat)
print(f'Test Loss: {evaluation[0]}, Test MAE: {evaluation[1]}')

# Make predictions
predictions = model.predict([branch_test, trunk_test])

# Inverse transform the predicted flows to original scale
predicted_flows = scaler_flow.inverse_transform(predictions.reshape(-1, 1)).reshape(-1, output_steps)
y_test_inverse = scaler_flow.inverse_transform(y_test.reshape(-1, 1)).reshape(-1, output_steps)

# Display predictions
results = pd.DataFrame({'Actual Flow': y_test_inverse.flatten(), 'Predicted Flow': predicted_flows.flatten()})
print(results.head())

# Calculate evaluation metrics
mae = mean_absolute_error(y_test_inverse.flatten(), predicted_flows.flatten())
mape = mean_absolute_percentage_error(y_test_inverse.flatten(), predicted_flows.flatten())
r2 = r2_score(y_test_inverse.flatten(), predicted_flows.flatten())
print(f'MAE: {mae}, MAPE: {mape}, R^2: {r2}')

# Plot loss
plt.figure(figsize=(10, 5))
plt.plot(losshistory.loss_train, label='Training Loss')
plt.plot(losshistory.loss_test, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')
plt.show()

# Plot rainfall-flow dual-axis chart for actual vs predicted flows along with aggregated rainfall
fig, ax1 = plt.subplots(figsize=(15, 10))

# Use first 100 data points for plotting
time_steps = np.arange(100)
aggregated_rainfall = scaler_rainfall.inverse_transform(X_test[:100, :, :-1].reshape(-1, X_test.shape[2] - 1)).sum(axis=1).reshape(100, -1).mean(axis=1)

# Plot aggregated rainfall as a bar chart, pointing downwards
ax1.bar(time_steps, -aggregated_rainfall, color='tab:green', alpha=0.5, label='Aggregated Rainfall')
ax1.invert_yaxis()

ax1.set_xlabel('Time Steps')
ax1.set_ylabel('Aggregated Rainfall (mm)')
ax1.legend(loc='upper right')
ax1.set_title('Rainfall and Flow Relationship')

# Plot actual and predicted flow
actual_flow = y_test_inverse[:100, 0]
predicted_flow = predicted_flows[:100, 0]

ax2 = ax1.twinx()
ax2.plot(time_steps, actual_flow, label='Actual Flow', color='blue', alpha=0.7)
ax2.plot(time_steps, predicted_flow, label='Predicted Flow', color='red', linestyle='dashed', alpha=0.7)

ax2.set_ylabel('Flow (m^3/s)')
ax2.legend(loc='upper left')

plt.show()